In [6]:
import numpy as np
import pandas as pd

def load_data(file_path):
    # Load the Excel file
    xls = pd.ExcelFile(file_path)
    # Load the "Games" sheet into a DataFrame
    games_data = pd.read_excel(xls, sheet_name='Games')
    return games_data

def process_games_data(df):
    # Add the "Gameweek" column
    df['Gameweek'] = 'Gameweek ' + (df.index + 1).astype(str)
    # Determine the winning team
    conditions = [
        df['Team A Goals'] > df['Team B Goals'],
        df['Team A Goals'] < df['Team B Goals'],
        df['Team A Goals'] == df['Team B Goals']
    ]
    choices = ['Team A', 'Team B', 'Draw']
    df['Winning Team'] = np.select(conditions, choices, default='Draw')
    # Calculate the total number of goals
    df['Total Goals'] = df['Team A Goals'] + df['Team B Goals']
    # Return the reordered DataFrame
    return df[['Date', 'Gameweek', 'Team A Goals', 'Team B Goals', 'Winning Team', 'Total Goals']]

# Load the games data and process it
games_data = load_data('FantaSPL_V2.3.xlsx')
processed_games_data = process_games_data(games_data)

# Load the "Points" sheet into a DataFrame
points_data = pd.read_excel('FantaSPL_V2.3.xlsx', sheet_name='Points')

# Count the number of players for each date
player_counts_per_date = points_data.groupby('Date').agg({'Player': 'nunique'}).reset_index()
player_counts_per_date.columns = ['Date', 'Number of Players']

# Merge the player counts with the processed_games_data dataframe based on the Date column
merged_data = pd.merge(processed_games_data, player_counts_per_date, on='Date', how='left')


def determine_game_type(row):
    if row['Number of Players'] <= 10:
        return '5-a-side'
    elif row['Number of Players'] <= 16:
        return '7-a-side'
    else:
        return '11-a-side'

merged_data['Game Type'] = merged_data.apply(determine_game_type, axis=1)

# Load the "Parameters" sheet into a DataFrame
parameters_data = pd.read_excel('FantaSPL_V2.3.xlsx', sheet_name='Parameters')

# Extract participation points for each game type based on the new structure
participation_points = {
    '5-a-side': parameters_data.loc[parameters_data['Parameter'] == 'Participation', '5-a-side'].values[0],
    '7-a-side': parameters_data.loc[parameters_data['Parameter'] == 'Participation', '7-a-side'].values[0],
    '11-a-side': parameters_data.loc[parameters_data['Parameter'] == 'Participation', '11-a-side'].values[0]
}

# Creating a new "Points" table from the "Points" sheet
new_points_data = points_data.copy()

# Merging with merged_data to get the game type for each player's participation
new_points_data = pd.merge(new_points_data, merged_data[['Date', 'Game Type', 'Team A Goals', 'Team B Goals', 'Winning Team', 'Total Goals']], on='Date', how='left')

# Adding the "Participation Points" column based on the game type
new_points_data['Participation Points'] = new_points_data['Game Type'].map(participation_points)

# Extracting points for each game outcome based on game type from the "Parameters" sheet
win_points = {
    '5-a-side': parameters_data.loc[parameters_data['Parameter'] == 'Win', '5-a-side'].values[0],
    '7-a-side': parameters_data.loc[parameters_data['Parameter'] == 'Win', '7-a-side'].values[0],
    '11-a-side': parameters_data.loc[parameters_data['Parameter'] == 'Win', '11-a-side'].values[0]
}

draw_points = {
    '5-a-side': parameters_data.loc[parameters_data['Parameter'] == 'Draw', '5-a-side'].values[0],
    '7-a-side': parameters_data.loc[parameters_data['Parameter'] == 'Draw', '7-a-side'].values[0],
    '11-a-side': parameters_data.loc[parameters_data['Parameter'] == 'Draw', '11-a-side'].values[0]
}

# Assuming points for loss is 0 for all game types
loss_points = {'5-a-side': 0, '7-a-side': 0, '11-a-side': 0}

def compute_game_outcome_points(row):
    # If the game was a draw
    if row['Winning Team'] == 'Draw':
        return draw_points[row['Game Type']]
    # If the player's team won
    elif row['Winning Team'] == row['Team']:
        return win_points[row['Game Type']]
    # If the player's team lost
    else:
        return loss_points[row['Game Type']]

# Computing the "Game Outcome Points" for each player using the updated table
new_points_data['Game Outcome Points'] = new_points_data.apply(compute_game_outcome_points, axis=1)
# Extracting points for goals scored based on game type from the "Parameters" sheet
goal_points = {
    '5-a-side': parameters_data.loc[parameters_data['Parameter'] == 'Goal', '5-a-side'].values[0],
    '7-a-side': parameters_data.loc[parameters_data['Parameter'] == 'Goal', '7-a-side'].values[0],
    '11-a-side': parameters_data.loc[parameters_data['Parameter'] == 'Goal', '11-a-side'].values[0]
}
# Computing the "Goal Points" for each player
new_points_data['Goal Points'] = new_points_data['Goals'] * new_points_data['Game Type'].map(goal_points)
# Extracting points for own goals scored based on game type from the "Parameters" sheet
own_goal_points = {
    '5-a-side': parameters_data.loc[parameters_data['Parameter'] == 'Own Goal', '5-a-side'].values[0],
    '7-a-side': parameters_data.loc[parameters_data['Parameter'] == 'Own Goal', '7-a-side'].values[0],
    '11-a-side': parameters_data.loc[parameters_data['Parameter'] == 'Own Goal', '11-a-side'].values[0]
}

# Computing the "Own Goal Points" for each player
new_points_data['Own Goal Points'] = new_points_data['Own Goals'] * new_points_data['Game Type'].map(own_goal_points)
# Extracting points for penalties based on game type from the "Parameters" sheet
penalty_points = {
    '5-a-side': parameters_data.loc[parameters_data['Parameter'] == 'Penalty', '5-a-side'].values[0],
    '7-a-side': parameters_data.loc[parameters_data['Parameter'] == 'Penalty', '7-a-side'].values[0],
    '11-a-side': parameters_data.loc[parameters_data['Parameter'] == 'Penalty', '11-a-side'].values[0]
}
# Computing the "Penalty Points" for each player
new_points_data['Penalty Points'] = new_points_data['Penalty'] * new_points_data['Game Type'].map(penalty_points)
# Extracting points SPL based on game type from the "Parameters" sheet
spl_points = {
    '5-a-side': parameters_data.loc[parameters_data['Parameter'] == 'SPL Bonus', '5-a-side'].values[0],
    '7-a-side': parameters_data.loc[parameters_data['Parameter'] == 'SPL Bonus', '7-a-side'].values[0],
    '11-a-side': parameters_data.loc[parameters_data['Parameter'] == 'SPL Bonus', '11-a-side'].values[0]
}
# Computing the "SPL Points" for each player
new_points_data['SPL Bonus'] = new_points_data['SPL Bonus'] * new_points_data['Game Type'].map(spl_points)

# Extracting points MVP based on game type from the "Parameters" sheet
mvp_points = {
    '5-a-side': parameters_data.loc[parameters_data['Parameter'] == 'MVP', '5-a-side'].values[0],
    '7-a-side': parameters_data.loc[parameters_data['Parameter'] == 'MVP', '7-a-side'].values[0],
    '11-a-side': parameters_data.loc[parameters_data['Parameter'] == 'MVP', '11-a-side'].values[0]
}
# Computing the "MVP Points" for each player
new_points_data['MVP'] = new_points_data['MVP'] * new_points_data['Game Type'].map(mvp_points)
# Extracting points Friend Referrals based on game type from the "Parameters" sheet
friend_referral_points = {
    '5-a-side': parameters_data.loc[parameters_data['Parameter'] == 'Friend Referral', '5-a-side'].values[0],
    '7-a-side': parameters_data.loc[parameters_data['Parameter'] == 'Friend Referral', '7-a-side'].values[0],
    '11-a-side': parameters_data.loc[parameters_data['Parameter'] == 'Friend Referral', '11-a-side'].values[0]
}

# Computing the "Goal Points" for each player
new_points_data['Friend Referral'] = new_points_data['Friend Referrals'] * new_points_data['Game Type'].map(friend_referral_points)
# Creating a mapping dictionary for player positions based on game type
position_mapping = {
    '11-a-side': 'Position 11-a-side',
    '7-a-side': 'Position 7-a-side',
    '5-a-side': 'Position 5 a-side'
}
#Adding postions based on matchtype
players_data = pd.read_excel('FantaSPL_V2.3.xlsx', sheet_name='Players')

# Function to get player position based on game type
def get_player_position(row):
    game_type = row['Game Type']
    player = row['Player']
    
    # Get the player position column name based on game type
    position_col = position_mapping[game_type]
    
    # Return the player position based on game type and player name
    return players_data.loc[players_data['Player'] == player, position_col].values[0] if player in players_data['Player'].values else np.nan

# Apply the function to get the player position and add it as a new column to the new_points_data DataFrame
new_points_data['Player Position'] = new_points_data.apply(get_player_position, axis=1)
new_points_data.head()
# Correcting the typo and re-extracting the necessary defensive parameters

# For 11-a-side matches
defense_score_11 = parameters_data.loc[parameters_data['Parameter'] == 'Defence Score', '11-a-side'].values[0]
goalkeeper_score_11 = parameters_data.loc[parameters_data['Parameter'] == 'Goalkeeper Score', '11-a-side'].values[0]

# For 7-a-side matches
defense_score_7 = parameters_data.loc[parameters_data['Parameter'] == 'Defence Score', '7-a-side'].values[0]
goalkeeper_score_7 = parameters_data.loc[parameters_data['Parameter'] == 'Goalkeeper Score', '7-a-side'].values[0]

# For 5-a-side matches
defense_score_5 = parameters_data.loc[parameters_data['Parameter'] == 'Defence Score', '5-a-side'].values[0]
goalkeeper_score_5 = parameters_data.loc[parameters_data['Parameter'] == 'Goalkeeper Score', '5-a-side'].values[0]

defense_score_11, goalkeeper_score_11, defense_score_7, goalkeeper_score_7, defense_score_5, goalkeeper_score_5
import math

def calculate_defensive_score(row):
    # Extracting necessary data from the row
    game_type = row['Game Type']
    player_position = row['Player Position']
    goals_conceded = row['Team A Goals'] if row['Team'] == 'Team B' else row['Team B Goals']
    
    # Define defensive score based on game type and player position
    if game_type == '11-a-side':
        if player_position == 'Goalkeeper':
            return defense_score_11 - goals_conceded + goalkeeper_score_11
        elif player_position == 'Defender':
            return defense_score_11 - goals_conceded
        elif player_position == 'Midfielder':
            return math.ceil(defense_score_11 - goals_conceded) / 2
        else:  # Forwards
            return 0
    
    elif game_type == '7-a-side':
        if player_position == 'Goalkeeper':
            return defense_score_7 - goals_conceded + goalkeeper_score_7
        elif player_position == 'Defensive Player':
            return defense_score_7 - goals_conceded
        else:  # Offensive Players
            return 0
    
    elif game_type == '5-a-side':
        if player_position == 'Goalkeeper':
            return defense_score_5 - goals_conceded + goalkeeper_score_5
        else:  # Outfield Players
            return defense_score_5 - goals_conceded

# Apply the function to the new_points_data DataFrame to compute the Defensive Score
new_points_data['Defensive Score'] = new_points_data.apply(calculate_defensive_score, axis=1)

# Display the first few rows of the updated DataFrame
new_points_data.tail()


,Date,Player,Team,Goals,Own Goals,SPL Bonus,MVP,Friend Referrals,Penalty,Game Type,...,Winning Team,Total Goals,Participation Points,Game Outcome Points,Goal Points,Own Goal Points,Penalty Points,Friend Referral,Player Position,Defensive Score
416,2023-07-12,Federico Paolucci,Team B,0,0,0,0,0,0,5-a-side,...,Team B,9,3,3,0,0,0,0,Outfield,6.0
417,2023-07-12,Cerro,Team B,3,0,0,0,0,0,5-a-side,...,Team B,9,3,3,3,0,0,0,Outfield,6.0
418,2023-07-12,Mazzu,Team B,2,0,0,0,0,0,5-a-side,...,Team B,9,3,3,2,0,0,0,Outfield,6.0
419,2023-07-12,Davide Ang,Team B,0,0,0,0,0,0,5-a-side,...,Team B,9,3,3,0,0,0,0,Outfield,6.0
420,2023-07-12,Andrea Scalambra,Team B,0,0,0,0,0,0,5-a-side,...,Team B,9,3,3,0,0,0,0,Outfield,6.0


Midfieled Score Calculation:

For 11-a-side matches:

Goalkeepers: no points
Defenders: no points
Midfielders: Goal difference
Forwards: Goal difference/2 round up


For 7-a-side matches:

Goalkeepers: no points
Defensive: no points
Offensive: Goal difference


For 5-a-side matches:

Goalkeepers: no points
Outfield: Goal difference

In [7]:
def calculate_midfield_score(row):
    # Extracting necessary data from the row
    game_type = row['Game Type']
    player_position = row['Player Position']
    goals_conceded = row['Team A Goals'] if row['Team'] == 'Team B' else row['Team B Goals']
    goals_scored = row['Team B Goals'] if row['Team'] == 'Team A' else row['Team A Goals']
    
    # Define midfield score based on game type and player position
    if game_type == '11-a-side':
        if player_position == 'Goalkeeper':
            return 0
        elif player_position == 'Defender':
            return 0
        elif player_position == 'Midfielder':
            return math.ceil(goals_scored - goals_conceded)
        else:  # Forwards
            return math.ceil(goals_scored - goals_conceded) / 2
    
    elif game_type == '7-a-side':
        if player_position == 'Goalkeeper':
            return 0
        elif player_position == 'Defensive Player':
            return 0
        else:  # Offensive Players
            return math.ceil(goals_scored - goals_conceded)
    
    elif game_type == '5-a-side':
        if player_position == 'Goalkeeper':
            return 0
        else:  # Outfield Players
            return math.ceil(goals_scored - goals_conceded)

# Apply the function to the new_points_data DataFrame to compute the Defensive Score
new_points_data['Midfield Score'] = new_points_data.apply(calculate_midfield_score, axis=1)

# Display the first few rows of the updated DataFrame
new_points_data.tail()

,Date,Player,Team,Goals,Own Goals,SPL Bonus,MVP,Friend Referrals,Penalty,Game Type,...,Total Goals,Participation Points,Game Outcome Points,Goal Points,Own Goal Points,Penalty Points,Friend Referral,Player Position,Defensive Score,Midfield Score
416,2023-07-12,Federico Paolucci,Team B,0,0,0,0,0,0,5-a-side,...,9,3,3,0,0,0,0,Outfield,6.0,0.0
417,2023-07-12,Cerro,Team B,3,0,0,0,0,0,5-a-side,...,9,3,3,3,0,0,0,Outfield,6.0,0.0
418,2023-07-12,Mazzu,Team B,2,0,0,0,0,0,5-a-side,...,9,3,3,2,0,0,0,Outfield,6.0,0.0
419,2023-07-12,Davide Ang,Team B,0,0,0,0,0,0,5-a-side,...,9,3,3,0,0,0,0,Outfield,6.0,0.0
420,2023-07-12,Andrea Scalambra,Team B,0,0,0,0,0,0,5-a-side,...,9,3,3,0,0,0,0,Outfield,6.0,0.0


In [8]:
def calculate_midfield_score(row):
    # Calculate the goal difference for the player's team
    if row['Team'] == 'Team A':
        goal_difference = row['Team A Goals'] - row['Team B Goals']
    else:
        goal_difference = row['Team B Goals'] - row['Team A Goals']

    # Define midfield score based on game type and player position
    if row['Game Type'] == '11-a-side':
        if row['Player Position'] == 'Goalkeeper' or row['Player Position'] == 'Defender':
            return 0
        elif row['Player Position'] == 'Midfielder':
            return goal_difference
        elif row['Player Position'] == 'Forward':
            return math.ceil(goal_difference / 2)  # Round up if it's a decimal
    
    elif row['Game Type'] == '7-a-side':
        if row['Player Position'] == 'Goalkeeper' or row['Player Position'] == 'Defensive Player':
            return 0
        elif row['Player Position'] == 'Offensive Player':
            return goal_difference
    
    elif row['Game Type'] == '5-a-side':
        if row['Player Position'] == 'Goalkeeper':
            return 0
        else:  # Outfield Players
            return goal_difference
    
    return 0  # Return 0 as default if none of the conditions above are met

# Apply the function to the new_points_data DataFrame to compute the Midfield Score
new_points_data['Midfield Score'] = new_points_data.apply(calculate_midfield_score, axis=1)

# Display the first few rows of the updated DataFrame
new_points_data.head()


,Date,Player,Team,Goals,Own Goals,SPL Bonus,MVP,Friend Referrals,Penalty,Game Type,...,Total Goals,Participation Points,Game Outcome Points,Goal Points,Own Goal Points,Penalty Points,Friend Referral,Player Position,Defensive Score,Midfield Score
0,2023-01-12,Andrea Limonta,Team B,0,0,3,0,0,0,7-a-side,...,31,4,0,0,0,0,0,Defensive,0.0,0
1,2023-01-12,Luca Stoppi,Team B,2,1,0,0,0,0,7-a-side,...,31,4,0,2,-2,0,0,Defensive,0.0,0
2,2023-01-12,Riccardo Ricci,Team B,0,0,0,0,0,0,7-a-side,...,31,4,0,0,0,0,0,Offensive,0.0,0
3,2023-01-12,Marco Taglio,Team B,3,0,0,0,0,0,7-a-side,...,31,4,0,3,0,0,0,Offensive,0.0,0
4,2023-01-12,Mazzu,Team B,7,0,0,0,0,0,7-a-side,...,31,4,0,7,0,0,0,Offensive,0.0,0
